In [ ]:
import torch
import torchvision
import os
import PIL
import pdb
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms,datasets, models
from PIL import Image
from skimage import io
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import shutil

In [ ]:
epochs = 35
batch = 32
lr = 1e-3
num_classes = 2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

train_losses =[]
tast_accuracy = []
trian_accuracy = []

In [ ]:
class Dataset(Dataset):
    def __init__(self,paths,transform = None):
        self.transform = transform
        items, labels=[],[]
        for dirc in os.listdir(paths):
            for img in os.listdir((paths+"/"+dirc)):
                item = os.path.join(paths+"/"+dirc,img)
                items.append(item)
                labels.append(dirc)
        self.items=items
        self.labels=labels
            
    def __len__(self):
          return len(self.items)

    def __getitem__(self, indax):
        image = self.items[indax]
        image = Image.open(image)

        if self.transform:
            image = self.transform(image)
        labal = int(self.labels[indax])
        return  torch.tensor(image), torch.tensor(labal)

In [ ]:
def show(data,name,idx):
    img,labal= next(iter(data))
    plt.imshow(img[idx])
    plt.title(name)
    plt.show()

In [ ]:
train_paths = "gan_img"
test_paths = "afhq\val"

data_transforms = {
    'train': transforms.Compose([transforms.Resize((250,250)),
        transforms.RandomHorizontalFlip(),transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([transforms.Resize((250,250)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

train_data = Dataset(train_paths,transform = data_transforms['train'])
train_dataloader = DataLoader(train_data, batch_size=batch, shuffle=True)

test_data = Dataset(test_paths,transform = data_transforms['test'])
test_dataloader = DataLoader(train_data, batch_size=batch, shuffle=False)


print("the longth for train: {}".format(len(train_dataloaders)))
print("the longth for test: {}".format(len(test_dataloaders)))
show(train_dataloader,"gan image",1)
show(test_dataloader,"real image",1)

In [ ]:
model = torchvision.models.googlenet(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(in_features=num_ftrs,out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512,out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128,out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32,out_features=num_classes)
) 

model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
for epoch in range(epochs):
    train_correct = 0 
    model.train()
    for img , labals in tqdm(train_dataloader):
        img = img.to(device)
        labals = labals.to(device)
        
        optimizer.zero_grad()
        outpots = model(img)
        loss = criterion(outpots,labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outpots,1)
        train_correct += (predicted == labels).sum().item()
        
        
    model.eval()
    with torch.no_grad():
        test_correct = 0
        for img,labels in test_dataloaders:
            img = img.to(device)
            labals = labals.to(device)
            
            outputs = model(img)                            
            _, predicted = torch.max(outputs, 1)
            test_correct += (predicted == labels).sum().item()

    train_acrr = 100 * train_correct/ len(train_data)
    tast_acrr = 100 * tast_correct/ len(tast_data)
    train_losses.append(loss.item())
    tast_accuracy.append(tast_acrr)
    trian_accuracy.append(train_acrr)
    
     print("epoch [{}/{}], loss: {:.4f}, train accuracy: {:.2f}%, test accuracy: {:.2f}%"
           .format(epoch+1,epochs,loss.item(),train_acrr,tast_acrr))
    
    if epoch == (epochs -1):
        plt.figure(figsize=(12,8))

        plt.plot(range(len(trian_accuracy)),trian_accuracy,label = "train accuracy")
        plt.plot(range(len(tast_accuracy)),tast_accuracy,label = "test accuracy")
        plt.plot(range(len(train_losses)),train_losses,label = "trian loss")

        plt.legend()
        plt.show()
        